In [0]:
sensible_data_identified = dataiku.Dataset("sensible_data_identified")
df = sensible_data_identified.get_dataframe()


In [0]:
from cryptography.fernet import Fernet

# Generate a key for encryption
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Define a function to encrypt text
def encrypt_text(text):
    return cipher_suite.encrypt(text.encode()).decode()

# Encrypt emails and IPs if present
def encrypt_sensitive_data(row):
    if pd.notnull(row['email_present']) or pd.notnull(row['ip_present']):
        if pd.notnull(row['email_present']):
            # Encrypt email
            row['encrypted_text'] = re.sub(row['email_present'], 
                                 lambda match: encrypt_text(match.group()), row['text'])
        if pd.notnull(row['ip_present']):
            # Encrypt IP
            row['encrypted_text'] = re.sub(row['ip_present'], 
                                 lambda match: encrypt_text(match.group()), row['text'])
    return row

# Apply the encryption function to the dataframe
df = df.apply(encrypt_sensitive_data, axis=1)


In [0]:
# Print the text of rows where email_present is not None and ip_present is not None
email_and_ip_present_rows = df[df['email_present'].notnull()| df['ip_present'].notnull()]
for index, row in email_and_ip_present_rows.iterrows():
    print(row['encrypted_text'])


In [0]:
def remove_sensitive_data(row):
    text = row['text']
    # Remove encrypted emails if email_present is not null
    text = text.replace(row['email_present'], '') if pd.notnull(row['email_present']) else text
    # Remove encrypted IPs if ip_present is not null
    text = text.replace(row['ip_present'], '') if pd.notnull(row['ip_present']) else text
    
    return text

# Create a new column 'text_without_sensible_data'
df['text_without_sensible_data'] = df.apply(remove_sensitive_data, axis=1)


In [0]:
# Print text without sensible data for a maximum of 5 rows where it's different from the original text
count = 0
for index, row in df.iterrows():
    if row['text'] != row['text_without_sensible_data']:
        print(row['text_without_sensible_data'])
        print()
        count += 1
        if count >= 5:
            break